In [1]:
import pandas as pd
import openpyxl as op

In [ ]:
sv = pd.read_excel("../data/документация/выгрузки/Стадии взысканий.xlsx")

Payment = pd.read_csv("../data/документация/выгрузки/np.csv")

# closedBills = pd.read_csv("../data/документация/выгрузки/zs.csv")
# closedBills["ДатаЗакрытия"] = pd.to_datetime(closedBills["ДатаЗакрытия"])

filter1 = pd.read_excel("../data/документация/выгрузки/объединенная классификация/Абонент Абонент Абонент.xlsx")
filter2 = pd.read_excel("../data/документация/выгрузки/объединенная классификация/Лица Призванные Наследовать.xlsx")
filter3 = pd.read_excel("../data/документация/выгрузки/объединенная классификация/Наниматель Жилого Помещения.xlsx")
filter4 = pd.read_excel("../data/документация/выгрузки/объединенная классификация/Юридические Лица.xlsx")
filter5 = pd.read_excel("../data/документация/выгрузки/объединенная классификация/Нежилое Помещение.xlsx")
total_filter = pd.concat([filter1, filter2, filter3, filter4, filter5])

df_ost = pd.read_csv("../data/документация/выгрузки/Остальное.csv")
df_ces = pd.read_csv("../data/документация/выгрузки/Цессии.csv")
df_mun = pd.read_csv("../data/документация/выгрузки/Мунжилье.csv")

def CategoryDZ(df):
    
    date = pd.to_datetime('2025-01-01')

    df['months_diff'] = (
        (date.year - df['Месяц долга'].dt.year) * 12
        + (date.month - df['Месяц долга'].dt.month)
    )

    df['years_diff'] = df['months_diff'] // 12

    def create_row(aLS, aNS, sum):
        return pd.DataFrame([{'КоличествоЛЦ': aLS, 'КоличествоДел': aNS, 'СуммаДолга': sum}])

    dfc = pd.DataFrame(columns=['КоличествоЛЦ', 'КоличествоДел', 'СуммаДолга'])

    # Отчетный месяц
    dfc = pd.concat([dfc, create_row(
                                    df.query('months_diff == 1')['Номер ЛС'].nunique(),
                                    df.query('months_diff == 1')['ROW_ID_Взыскание'].nunique(),
                                    df.query('months_diff == 1')['Сумма'].sum())])

    # от 2 до 3 месяцев после отчетного
    dfc = pd.concat([dfc, create_row(
                                    df.query('3 >= months_diff >= 2')['Номер ЛС'].nunique(),
                                    df.query('3 >= months_diff >= 2')['ROW_ID_Взыскание'].nunique(),
                                    df.query('3 >= months_diff >= 2')['Сумма'].sum())])

    # от 3 месяцев до 3 лет после отчетного
    dfc = pd.concat([dfc, create_row(
                                    df.query('months_diff > 3 and years_diff < 3')['Номер ЛС'].nunique(),
                                    df.query('months_diff > 3 and years_diff < 3')['ROW_ID_Взыскание'].nunique(),
                                    df.query('months_diff > 3 and years_diff < 3')['Сумма'].sum())])

    # свыше 3 лет
    dfc = pd.concat([dfc, create_row(
                                    df.query('years_diff >= 3')['Номер ЛС'].nunique(),
                                    df.query('years_diff >= 3')['ROW_ID_Взыскание'].nunique(),
                                    df.query('years_diff >= 3')['Сумма'].sum())])

    # не было оплат
    noPayment = df[~df['Счет'].isin(Payment)]

    dfc = pd.concat([dfc, create_row(
                                    noPayment['Номер ЛС'].nunique(),
                                    noPayment['ROW_ID_Взыскание'].nunique(),
                                    noPayment['Сумма'].sum())])
    
    # Свыше 3-х лет+все остальные периоды по этим (закрытым) л/с
    # closedBills['yearsdiff'] = (date.year - closedBills["ДатаЗакрытия"].dt.year) + \
    #                      ((date.month - closedBills["ДатаЗакрытия"].dt.month) / 12)
    # periodFilter = closedBills[closedBills["yearsdiff"] >= 3]

    allPeriods = df[df["ЛицевойЗакрыт"] == 1]

    allPeriods = allPeriods[allPeriods['Номер ЛС'].isin(df.query('years_diff >= 3')['Номер ЛС'])]

    dfc = pd.concat([dfc, create_row(
                                    allPeriods['Номер ЛС'].nunique(),
                                    allPeriods['ROW_ID_Взыскание'].nunique(),
                                    allPeriods['Сумма'].sum())])
    
    # ДЗ менее 3000 р 
    small_sum = df.groupby(['Номер ЛС', 'ROW_ID_Взыскание'])['Сумма'].sum().reset_index()
    small_sum = small_sum[small_sum['Сумма'] < 3000]
    
    # small_sum_filter = df[df['Номер ЛС'].isin(small_sum['Номер ЛС'])]
    
    dfc = pd.concat([dfc, create_row(
                                    small_sum['Номер ЛС'].nunique(),  
                                    small_sum['ROW_ID_Взыскание'].nunique(),            
                                    small_sum['Сумма'].sum())]) 
    
    # Без ФИО, Лица призванные наследовать, Юридические лица, Нежилое помещение, Наниматель жилого помещения, Несовершеннолетние
    dfcopy_filtered = df[df["ФИО"].isin(total_filter["ФИО"])]

    dfc = pd.concat([dfc, create_row(
                                    dfcopy_filtered['Номер ЛС'].nunique(),
                                    dfcopy_filtered['ROW_ID_Взыскание'].nunique(),
                                    dfcopy_filtered['Сумма'].sum())])
    return dfc


def Category(df, start_row):
    dfcopy = df.copy(deep=True)
    dfcopy = pd.merge(dfcopy, sv, how="left")

    def updateExcel(df, start_row):
        wb = op.load_workbook('./отчеты/Отчет ежемесячный КT2(копия).xlsx')
        wba = wb.active
        start_col = 6 

        for row_index, (index, row) in enumerate(df.iterrows(), start=start_row):  
            for col_index, (col_name, cell_value) in enumerate(row.items(), start=start_col):
                cell = wba.cell(row=row_index, column=col_index)
                cell.value = cell_value

        wb.save('./отчеты/Отчет ежемесячный КT2(копия).xlsx')

    dfcopy['Сумма'] = pd.to_numeric(dfcopy['Сумма'].replace(',', '.', regex=True).astype(float))
    dfcopy['Месяц долга'] = pd.to_datetime(dfcopy['Месяц долга'])

    # Досудебная
    Dosud = dfcopy[(dfcopy["Общая стадия"] == "Досубедная от месяца") | dfcopy["Фаза"].isna()]

    # Судебная
    Sud = dfcopy[dfcopy["Общая стадия"] == "Суд"]

    # ФССП
    Fssp = dfcopy[(dfcopy["Общая стадия"] == "ФССП")]

    total_df = pd.concat([CategoryDZ(Dosud), CategoryDZ(Sud), CategoryDZ(Fssp)], axis=1)

    updateExcel(total_df, start_row)


Category(df_ost, 17)
Category(df_ces, 27)
Category(df_mun, 37)

C:\Users\n.kostryukov\AppData\Local\Temp\ipykernel_38120\2422738237.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['months_diff'] = (
C:\Users\n.kostryukov\AppData\Local\Temp\ipykernel_38120\2422738237.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['years_diff'] = df['months_diff'] // 12
C:\Users\n.kostryukov\AppData\Local\Temp\ipykernel_38120\2422738237.py:36: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will 

In [45]:
df_ost[df_ost['Номер ЛС'] == 59].sort_values(by='Фаза')

,Unnamed: 0,Сумма,Номер ЛС,Счет,Месяц долга,Наименование,Фаза,Дата постановки на фазу,ДатНачДолга,ДатКнцДолга,НомерДела,НебылоОплат,ЛицевойЗакрыт,ROW_ID_Организации,ФИО,Дата рождения,Дата смерти,ROW_ID_Взыскание
402379,402379,210.49,59,670080,2022-04-01,"ООО ""Концессии теплоснабжения"" НСУ Тракторозав...",Исполнительное производство Сдача в ФССП,2023-04-20 00:00:00,2021-03-01,2022-04-01,2.148314e+09,0,0,5570,Таторина Оксана Михайловна,1989-02-14 00:00:00,NaN,1066951.0
77515,77515,1913.82,59,670080,2023-01-01,"ООО ""Концессии теплоснабжения"" НСУ Тракторозав...",Рассмотрение (абонентский отдел),2024-11-27 00:00:00,2022-11-01,2024-10-01,2.148794e+09,0,0,5570,Таторина Оксана Михайловна,1989-02-14 00:00:00,NaN,1562572.0
129271,129271,1325.32,59,670080,2024-10-01,"ООО ""Концессии теплоснабжения"" НСУ Тракторозав...",Рассмотрение (абонентский отдел),2024-11-27 00:00:00,2022-11-01,2024-10-01,2.148794e+09,0,0,5570,Таторина Оксана Михайловна,1989-02-14 00:00:00,NaN,1562572.0
186746,186746,1936.14,59,670080,2023-11-01,"ООО ""Концессии теплоснабжения"" НСУ Тракторозав...",Рассмотрение (абонентский отдел),2024-11-27 00:00:00,2022-11-01,2024-10-01,2.148794e+09,0,0,5570,Таторина Оксана Михайловна,1989-02-14 00:00:00,NaN,1562572.0
192341,192341,1922.68,59,670080,2022-12-01,"ООО ""Концессии теплоснабжения"" НСУ Тракторозав...",Рассмотрение (абонентский отдел),2024-11-27 00:00:00,2022-11-01,2024-10-01,2.148794e+09,0,0,5570,Таторина Оксана Михайловна,1989-02-14 00:00:00,NaN,1562572.0
365497,365497,1936.14,59,670080,2023-04-01,"ООО ""Концессии теплоснабжения"" НСУ Тракторозав...",Рассмотрение (абонентский отдел),2024-11-27 00:00:00,2022-11-01,2024-10-01,2.148794e+09,0,0,5570,Таторина Оксана Михайловна,1989-02-14 00:00:00,NaN,1562572.0
480666,480666,1936.14,59,670080,2024-01-01,"ООО ""Концессии теплоснабжения"" НСУ Тракторозав...",Рассмотрение (абонентский отдел),2024-11-27 00:00:00,2022-11-01,2024-10-01,2.148794e+09,0,0,5570,Таторина Оксана Михайловна,1989-02-14 00:00:00,NaN,1562572.0
504821,504821,1913.55,59,670080,2023-02-01,"ООО ""Концессии теплоснабжения"" НСУ Тракторозав...",Рассмотрение (абонентский отдел),2024-11-27 00:00:00,2022-11-01,2024-10-01,2.148794e+09,0,0,5570,Таторина Оксана Михайловна,1989-02-14 00:00:00,NaN,1562572.0
582884,582884,1936.14,59,670080,2023-10-01,"ООО ""Концессии теплоснабжения"" НСУ Тракторозав...",Рассмотрение (абонентский отдел),2024-11-27 00:00:00,2022-11-01,2024-10-01,2.148794e+09,0,0,5570,Таторина Оксана Михайловна,1989-02-14 00:00:00,NaN,1562572.0
599650,599650,1745.87,59,670080,2022-11-01,"ООО ""Концессии теплоснабжения"" НСУ Тракторозав...",Рассмотрение (абонентский отдел),2024-11-27 00:00:00,2022-11-01,2024-10-01,2.148794e+09,0,0,5570,Таторина Оксана Михайловна,1989-02-14 00:00:00,NaN,1562572.0


In [15]:
def create_row(aLS, aNS, sum):
    return pd.DataFrame([{'КоличествоЛЦ': aLS, 'КоличествоДел': aNS, 'СуммаДолга': sum}])

In [19]:
date = pd.to_datetime('2025-01-01')

df_ost['Месяц долга'] = pd.to_datetime(df_ost['Месяц долга'])
df_ost['months_diff'] = (
    (date.year - df_ost['Месяц долга'].dt.year) * 12
    + (date.month - df_ost['Месяц долга'].dt.month)
)

df_ost['years_diff'] = df_ost['months_diff'] // 12

def create_row(aLS, aNS, sum):
    return pd.DataFrame([{'КоличествоЛЦ': aLS, 'КоличествоДел': aNS, 'СуммаДолга': sum}])

dfc = pd.DataFrame(columns=['КоличествоЛЦ', 'КоличествоДел', 'СуммаДолга'])

# Отчетный месяц
dfc = pd.concat([dfc, create_row(
                                df_ost.query('months_diff == 0')['Номер ЛС'].nunique(),
                                df_ost.query('months_diff == 0')['ROW_ID_Взыскание'].nunique(),
                                df_ost.query('months_diff == 0')['Сумма'].sum())])

C:\Users\n.kostryukov\AppData\Local\Temp\ipykernel_39168\1415189770.py:17: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  dfc = pd.concat([dfc, create_row(


In [36]:
df1 = df_ost[(df_ost['Месяц долга'] == "2024-11-01") | (df_ost['Месяц долга'] == "2024-10-01")]
df1['Сумма'] = pd.to_numeric(df1['Сумма'].replace(',', '.', regex=True).astype(float))
df1['Сумма'].sum()

C:\Users\n.kostryukov\AppData\Local\Temp\ipykernel_39168\67548795.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['Сумма'] = pd.to_numeric(df1['Сумма'].replace(',', '.', regex=True).astype(float))


np.float64(181218372.05)

In [50]:
small_sum = df_ost.groupby(['Номер ЛС', 'Счет', 'ROW_ID_Взыскание'])['Сумма'].sum().reset_index()
small_sum

,Номер ЛС,Счет,ROW_ID_Взыскание,Сумма
0,59,670080,1066951.0,210.49
1,59,670080,1562572.0,21238.19
2,62,670096,738920.0,2654.32
3,62,670096,929532.0,23549.06
4,62,670096,1180858.0,16284.56
...,...,...,...,...
71088,4720955914,3287489,664963.0,1809.44
71089,4720955914,3287489,664965.0,10697.14
71090,4720955914,3287489,1165863.0,261.58
71091,4720955914,3287489,1211911.0,12383.75


In [55]:
small_sum = small_sum[small_sum['Сумма'] < 3000]
small_sum['Номер ЛС'].nunique()
small_sum['Сумма'].sum()

np.float64(6922162.3100000005)

In [ ]:
small_sum_filter = df_ost[df_ost['Номер ЛС'].isin(small_sum['Номер ЛС'])]

In [48]:
small_sum_filter['Сумма'].sum()

np.float64(204163370.35999998)

In [8]:
df_ost['Сумма'].sum()

np.float64(1567113874.6399994)